# Data Wrangling<a id='Data_Wrangling'>

This notebook will take movie data and clean it to get it ready for Exploratory Data Analysis.  

Data Source: https://www.kaggle.com/datasets/benjameeper/movie-violencesexprofanity-data?resource=download

## Table of Contents<a id='Table_of_Contents'>

* [Data Wrangling](#Data_Wrangling)
  * [Table of Contents](#Table_of_Contents)
  * [Imports](#Imports)
  * [Functions/Constants](#Functions)
      * [Functions](#Functions_sub)
      * [Constants](#Constants)
  * [Load The Data](#Load_the_Data)
  * [Clean the Data](#Clean_the_Data)
      * [tags](#tags)
      * [movies](#movies)
      * [movie_tags](#movie_tags)
* [Save the Data](#Save_the_Data)

## Imports <a id='Imports'>

First we will import all the necessary packages we will need. 

In [1]:
import pandas as pd
from collections import Counter
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## Functions/Constants <a id='Functions'>

Next we will define all of our functions and constants that we will need for this notebook. 

### Functions <a id='Functions_sub'>

Here are our functions that we will be using. They are used to find similar studios and cleanup duplicate values. 

In [2]:
# Function to find similar studio names
def find_similar_studios(target_studio, all_studios, threshold=95):
    '''This is a function to find similarly named studios.
    Uses a fuzzy score of 95 and will return all the studios and their similarities.'''
    
    similar_studios = process.extract(target_studio, all_studios, scorer=fuzz.partial_ratio)
    return [(studio, score) for studio, score in similar_studios if score >= threshold]

In [3]:
# Apply a function to each row to keep one instance of each value and replace the rest with None
def clean_duplicates(row):
    '''This function is used to cleanup the rows of a dataframe to have no duplicates and replaces them with None. 
    Is used in an apply method on a dataframe for each row.'''
    
    seen_values = set()
    new_row = []
    for item in row:
        if item is not None and item not in seen_values:
            seen_values.add(item)
            new_row.append(item)
        else:
            new_row.append(None)
    return new_row

### Constants <a id='Constants'>

Next we will define our constants. This will contain our mapping of all the studios that had duplicates of somekind. This was found later in Exploratory Data Analysis, but the changes need to happen within our cleaning steps. I went through and mapped out in a dictionary what the name of the studio currently is, and what studio it will be included in to create a consistent studio name. 

In [4]:
#Map duplicate studios so they are more consistent across movies. 
studio_mapping = {
    'Amazon': 'Amazon Studios',
    'Paramount Pictures': 'Paramount', 
    'Paramount Players':'Paramount', 
    'Paramount Vantage':'Paramount', 
    'Paramount Pictures Corporation':'Paramount',
    'Paramount Pictures International':'Paramount',
    'Don Simpson/Jerry Bruckheimer Films':'Jerry Bruckheimer Films',
    'Summit entertainment':'Summit Entertainment',
    'Universal PIctures':'Universal Pictures',
    'Universal 1440 Entertainment':'Universal Pictures',
    'Universal':'Universal Pictures',
    'verture Films':'Overture Films',
    'Participant Media':'Participant',
    'di Bonaventura Pictures':'Di Bonaventura Pictures',
    'Skydance Sports':'Skydance', 
    'Skydance Media': 'Skydance', 
    'Skydance Animation':'Skydance',
    'Pioneer Stilking Films':'Stillking Films',
    '87Eleven':'87Eleven Productions',
    'Apple':'Apple Studios',
    'Apple Inc.':'Apple Studios',
    'dentsu':'Dentsu',
    'DENTSU':'Dentsu',
    'Lord Miller':'Lord Miller Productions',
    'Blur Studio':'Blur Studios',
    'Netflix Animation':'Netflix',
    'Netflix Studios':'Netflix',
    'netflix':'Netflix',
    'Netflix International Pictures':'Netflix',
    'Lions Gate Entertainment':'Lions Gate',
    'Lions Gate Films Inc':'Lions Gate',
    'Lions Gate Films':'Lions Gate',
    'Lionsgate':'Lions Gate',
    'Carnival Films & Television':'Carnival Films',
    'Sony Pictures':'Sony',
    'Sony Pictures Animation':'Sony',
    'Sony Pictures Television':'Sony',
    'Sony Pictures Imageworks':'Sony',
    'Sony Pictures Classics':'Sony',
    'Sony Music Entertainment (Japan)':'Sony',
    'Metro-Goldwyn-Mayer (MGM)':'MGM',
    'MGM Television':'MGM',
    'Columbia Pictures':'Columbia',
    'Columbia Pictures Industries':'Columbia',
    'DreamWorks Animation':'DreamWorks',
    'DreamWorks Pictures':'DreamWorks',
    'Dreamworks Pictures':'DreamWorks',
    'DreamWorks II Distribution Co., LLC':'DreamWorks',
    'Dreamworks':'DreamWorks',
    'DreamWorks Animation L.L.C.':'DreamWorks',
    'IAC Films [US]':'IAC Films',
    'Eclectic pictures':'Eclectic Pictures',
    'Thunder Road':'Thunder Road Pictures',
    'StudioCanal':'Canal+',
    'Studio Canal':'Canal+',
    'Atresmedia':'Atresmedia Cine',
    'Anchor Bay Entertainment':'Anchor Bay',
    'Nolita':'Nolita Cinema',
    'Highland Film Group (HFG)':'Highland Film Group',
    'Global Entertainment Productions GmbH & Company Medien KG':'Global Entertainment Productions',
    'Focus Features':'Focus',
    'Focus Features International (FFI)':'Focus',
    'Focus World':'Focus',
    'Motion Picture Corporation of America (MPCA)':'Motion Picture Corporation of America',
    'Cinesite Animation':'Cinesite',
    'Mockingbird Pirctures':'Mockingbird Pictures',
    'EO':'Levitate Film',
    'Tri-Star Pictures, inc':'Tri Star',
    'BBC Film':'BBC',
    'BBC Studios':'BBC',
    'BBC Films':'BBC',
    'IFC Films':'IFC',
    'IFC Midnight':'IFC',
    'Street Entertainment':'10th Street Entertainment',
    'The Weinstein Company':'Weinstein Company',
    'Pure Flix Entertainment':'Pure Flix',
    'Pathe':'Pathe Productions',
    'Radar Pictures Inc.':'Radar Pictures',
    'Danjaq':'Danjaq, LLC',
    'Film4':'Film4 Productions',
    'uMedia':'Umedia',
    'The Hallmark Channel':'Hallmark',
    'Hanway Films':'HanWay Films',
    'Bona Fide':'Bona Fide Productions',
    'VidAngel Studios':'Angel Studios',
    'Northern Lights Film':'Northern Lights Films',
    'Pantelion Film':'Pantelion Films'
}

## Load the Data <a id='Load_the_Data'>

Now let's load the data for our three dataframes. 

In [5]:
tags = pd.read_csv('data/raw_data/tags.csv')
movies = pd.read_csv('data/raw_data/movies.csv')
movie_tags = pd.read_csv('data/raw_data/movie_tags.csv')

## Clean the Data <a id='Clean_the_Data'>

It's time to start our cleaning steps. Let's start with the tags dataframe. 

### tags <a id='tags'>

We will first view the dataframe and see what it looks like and if anything needs to be cleaned. 

In [6]:
pd.set_option('display.max_colwidth', None)
tags

,category,tag_name,title,description
0,language,profanity,Profanity,NaN
1,language,blasphemy,Blasphemy,NaN
2,language,sexual_reference,Sexual References and Innuendos,"Any references or jokes about sex, flirting, innuendos, etc."
3,language,childish_language,Childish Language,"Generally, things you would not want your 3-year-old to repeat."
4,language,racial_slurs,Racial Slurs and Bigoted Language,"Racist, sexist, and/or discriminatory language in any form."
5,violence,non_graphic,Non-Graphic,Violence without blood.
6,violence,graphic,Graphic,Violence with blood or breaking bones.
7,violence,disturbing_images,Disturbing Images,"Dead bodies, severed body parts, or object protruding from body"
8,violence,gore,Gore,"Gore, bloody guts, bloody severed body parts."
9,violence,violence_implied,Implied Violence,The violence is not seen on screen. Graphic descriptions or details of a violent act.


We see there are a couple of tags of NaN as the description. However, we won't be using the description in any analysis so we will leave them as NaN. 

I will be adding a new row to the tags for 'Other' to be able to account for anything that is not already listed in this tag list. It won't need to be used in this analysis, but wanted to include it in case it was ever needed for any new data that did not fall under any of these tags. 

In [7]:
#Add an other tag_name under the other category to be able to capture anything that does not already have a specific tag.
new_row = pd.DataFrame({'category': ['other'], 'tag_name': ['other'],'title': ['Other']})

tags = pd.concat([tags, new_row], ignore_index=True)

tags


,category,tag_name,title,description
0,language,profanity,Profanity,NaN
1,language,blasphemy,Blasphemy,NaN
2,language,sexual_reference,Sexual References and Innuendos,"Any references or jokes about sex, flirting, innuendos, etc."
3,language,childish_language,Childish Language,"Generally, things you would not want your 3-year-old to repeat."
4,language,racial_slurs,Racial Slurs and Bigoted Language,"Racist, sexist, and/or discriminatory language in any form."
5,violence,non_graphic,Non-Graphic,Violence without blood.
6,violence,graphic,Graphic,Violence with blood or breaking bones.
7,violence,disturbing_images,Disturbing Images,"Dead bodies, severed body parts, or object protruding from body"
8,violence,gore,Gore,"Gore, bloody guts, bloody severed body parts."
9,violence,violence_implied,Implied Violence,The violence is not seen on screen. Graphic descriptions or details of a violent act.


We see that the Other row has been added, so we should be good to proceed to the next dataframe, movies. 

### movies <a id='movies'>

Let's first view the movies dataframe and see what it looks like. 

In [8]:
pd.reset_option('display.max_colwidth')
movies

,imdb_id,name,title_main,title_subscript,year,mpaa_rating,duration_sec,studio
0,tt11274492,The Out-Laws,The Out-Laws,NaN,2023,R,5700,Happy Madison Productions
1,tt12263384,Extraction 2,Extraction 2,NaN,2023,R,7380,Filmhaus Films|AGBO
2,tt16419074,Air,Air,NaN,2023,R,6720,Mandalay Pictures|Amazon Studios|Skydance Spor...
3,tt14400246,Bird Box Barcelona,Bird Box Barcelona,NaN,2023,TV-MA,7440,Nostromo Pictures|Bluegrass Films|Chris Morgan...
4,tt1745960,Top Gun: Maverick,Top Gun: Maverick,NaN,2022,PG-13,7860,Paramount|Jerry Bruckheimer Films|Don Simpson/...
...,...,...,...,...,...,...,...,...
1729,tt6902676,Guns Akimbo,Guns Akimbo,NaN,2020,R,5700,Ingenious Media|Occupant Films|Four Knights Fi...
1730,tt3813310,Cop Car,Cop Car,NaN,2015,R,5280,Universal
1731,tt2091935,Mr. Right,Mr. Right,NaN,2016,R,5700,Focus World
1732,tt13372794,The Manor,The Manor,NaN,2021,TV-MA,4860,Amazon Studios|Blumhouse Television


Some potential things we will need to look at are the null values in the title_subscript and see how that relates to the title_main column. We'll also want to look for any dupliacte movies, movies that have the same name, as well as explore the studios for any duplicates/changes to make. Let's see the datatypes of the columns. 

In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1734 entries, 0 to 1733
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   imdb_id          1734 non-null   object
 1   name             1734 non-null   object
 2   title_main       1734 non-null   object
 3   title_subscript  32 non-null     object
 4   year             1734 non-null   int64 
 5   mpaa_rating      1734 non-null   object
 6   duration_sec     1734 non-null   int64 
 7   studio           1734 non-null   object
dtypes: int64(2), object(6)
memory usage: 108.5+ KB


We see that all the datatypes are what we would expect, so no adjustments are needed there. We also see the only column with missing values is that title_subscript column. Let's find all movies that have a title subscript populated. 

In [10]:
subscript_movies = movies[movies.title_subscript > ""]

subscript_movies

,imdb_id,name,title_main,title_subscript,year,mpaa_rating,duration_sec,studio
39,tt1229238,Mission: Impossible IV: Ghost Protocol,Mission: Impossible IV,Ghost Protocol,2011,PG-13,7980,Paramount Pictures
56,tt2368619,Bastille Day: The Take,Bastille Day,The Take,2016,R,5520,StudioCanal
76,tt0103064,Terminator 2: Judgment Day,Terminator 2,Judgment Day,1991,R,8220,Lions Gate Entertainment
177,tt1590193,The Commuter: Lives are on the line,The Commuter,Lives are on the line,2018,PG-13,6240,StudioCanal
244,tt9742794,The Vault: aka Way Down,The Vault,aka Way Down,2021,R,7080,Telecinco Cinema|Think Studio|Saban Films|Ciud...
245,tt2250912,Spider-Man: Homecoming: Theatrical Version,Spider-Man: Homecoming,Theatrical Version,2017,PG-13,7980,Columbia Pictures Industries
290,tt4106376,American Renegades: (aka Renegades),American Renegades,(aka Renegades),2017,PG-13,6300,Studio Babelsberg
345,tt4514646,Tim Timmerman: Hope of America,Tim Timmerman,Hope of America,2017,PG-13,5916,VidAngel Studios
412,tt5536736,The Meyerowitz Stories: New and Selected,The Meyerowitz Stories,New and Selected,2017,TV-MA,6720,IAC Films
422,tt9657544,Jim Gaffigan: Quality Time,Jim Gaffigan,Quality Time,2019,PG-13,4500,Comedy Dynamics


Looking at the list, it appears the title_subtitle column takes the name of the movie and if there exists a ':', it substrings the title where anything before the ':' is the title and anything after is the subtitle. Let's now check to see if any duplicate title_mains exist in the original movies dataframe.

In [11]:
#The resulting check_dups_df DataFrame will contain all rows from the movies DataFrame where the 'title_main' 
#value matches one of the unique 'title_main' values in subscript_movies, which effectively shows you which 
#entries in movies have duplicates based on the 'title_main' column.

check_dups = []

for title in subscript_movies['title_main']:
    result = movies[movies['title_main'] == title]
    check_dups.append(result)

check_dups_df = pd.concat(check_dups, ignore_index=True).sort_values(by='name')
check_dups_df

,imdb_id,name,title_main,title_subscript,year,mpaa_rating,duration_sec,studio
23,tt5503688,6 Below: Miracle on the Mountain,6 Below,Miracle on the Mountain,2017,PG-13,5820,Dune Entertainment
6,tt4106376,American Renegades: (aka Renegades),American Renegades,(aka Renegades),2017,PG-13,6300,Studio Babelsberg
1,tt2368619,Bastille Day: The Take,Bastille Day,The Take,2016,R,5520,StudioCanal
28,tt4230700,Complete Unknown: Identities,Complete Unknown,Identities,2016,R,5460,Parts and Labor
26,tt3317234,"Game Over, Man!: Game Over, Man!","Game Over, Man!","Game Over, Man!",2018,TV-MA,6060,Point Grey Pictures
12,tt5052448,Get Out: (with alternate ending),Get Out,(with alternate ending),2017,R,6240,Universal Pictures
24,tt10665338,Halloween Kills: Extended Cut,Halloween Kills,Extended Cut,2021,TV-MA,6300,Lionsgate
20,tt5726086,Insidious: The Last Key,Insidious,The Last Key,2018,PG-13,6180,Stage 6 Films
19,tt2908446,Insurgent: Insurgent,Insurgent,Insurgent,2015,PG-13,7140,Universal
9,tt9657544,Jim Gaffigan: Quality Time,Jim Gaffigan,Quality Time,2019,PG-13,4500,Comedy Dynamics


In [12]:
check_dups_df[check_dups_df.duplicated()]

,imdb_id,name,title_main,title_subscript,year,mpaa_rating,duration_sec,studio


Not seeing any duplicates for all titles that exist so seems like title_main and title_subscript may not be needed since the name column should have the accurate movie title with no duplicates. Let's now check for duplicates for all movies. 

In [13]:
movies[movies.duplicated()]

,imdb_id,name,title_main,title_subscript,year,mpaa_rating,duration_sec,studio


No current duplicates. Let's drop those two columns since all we will need is the name of the movie. 

In [14]:
movies.drop(['title_main','title_subscript'], axis=1, inplace=True)

Another thing to check is whether there are any movies that are in our movies dataframe, but do not have any corresponding tags in our movie_tags dataframe. If movies exist like that, we won't be able to use them for our analysis as the information regarding the content of the movie is not available. Let's first check and see how many unique imdb_id's are in the movie_tags dataframe. 

In [15]:
movie_tags['imdb_id'].nunique()

1729

We see there are 1729 unique id's. Let's now compare that to the movies dataframe and see if they match. 

In [16]:
movies['imdb_id'].nunique()

1734

We see there are 1734 total movies listed, so there are 5 movies that don't have any movie tags. Let's find the id's that are missing as we will need to drop those rows from our movies dataframe. 

In [17]:
# Get unique imdb_id values from both dataframes
unique_imdb_ids_cleaned = movies['imdb_id'].unique()
unique_imdb_ids_tags = movie_tags['imdb_id'].unique()

# Find imdb_id values that are in cleaned_movies but not in movie_tags
missing_imdb_ids = movies[~movies['imdb_id'].isin(unique_imdb_ids_tags)]
missing_imdb_ids

,imdb_id,name,year,mpaa_rating,duration_sec,studio
538,tt0317198,Bridget Jones: The Edge of Reason,2004,R,6456,Lions Gate Entertainment
575,tt1772264,Stonehearst Asylum,2014,PG-13,6720,Icon Productions
619,tt0375679,Crash,2005,R,6720,Lions Gate Entertainment
854,tt9702698,Timeless Love,2019,PG,5100,Candlelight Media Group
1364,tt1261954,Take Me Home,2011,PG-13,5820,Good Folk Films


We have found the 5 movies that don't have any tags. We see their ratings are different including a couple of R ratings, so there definitely should be at least 1 tag associated to these movies. Since there are not, we will drop these rows from our cleaned movies dataframe. 

In [18]:
# List of imdb_id values to drop
drop_ids = ['tt0317198',
'tt1772264',
'tt0375679',
'tt9702698',
'tt1261954']

# Filter rows to keep only those not in the drop_ids list
movies = movies[~movies['imdb_id'].isin(drop_ids)]

movies

,imdb_id,name,year,mpaa_rating,duration_sec,studio
0,tt11274492,The Out-Laws,2023,R,5700,Happy Madison Productions
1,tt12263384,Extraction 2,2023,R,7380,Filmhaus Films|AGBO
2,tt16419074,Air,2023,R,6720,Mandalay Pictures|Amazon Studios|Skydance Spor...
3,tt14400246,Bird Box Barcelona,2023,TV-MA,7440,Nostromo Pictures|Bluegrass Films|Chris Morgan...
4,tt1745960,Top Gun: Maverick,2022,PG-13,7860,Paramount|Jerry Bruckheimer Films|Don Simpson/...
...,...,...,...,...,...,...
1729,tt6902676,Guns Akimbo,2020,R,5700,Ingenious Media|Occupant Films|Four Knights Fi...
1730,tt3813310,Cop Car,2015,R,5280,Universal
1731,tt2091935,Mr. Right,2016,R,5700,Focus World
1732,tt13372794,The Manor,2021,TV-MA,4860,Amazon Studios|Blumhouse Television


In [19]:
movies['imdb_id'].nunique()

1729

We see that the number of unique id's now match between the two dataframes. 

Now that we have our full subset of movies, let's take a closer look at the sudio column. I found later in my analysis that there was an issue with commas in some of the studio names. For example, the movie White Boy Rick includes 'Protozoa Pictures, Studio 8' as one of the studios, but that comma really should have been a | to separate them. Here I am updating the values properly so it won't cause issues in the analysis later on. 

In [20]:
pd.set_option('display.max_colwidth', None)
comma_studios = movies[movies['studio'].str.contains(',', na=False)]
comma_studios

,imdb_id,name,year,mpaa_rating,duration_sec,studio
264,tt1817273,The Place Beyond the Pines,2013,R,8400,"Kimmel Distribution, LLC"
288,tt4633694,Spider-Man: Into the Spider-Verse,2018,PG,7020,"Columbia Pictures, Sony Pictures, Sony Pictures Animation"
291,tt0071853,Monty Python and the Holy Grail,1975,PG,5460,"National Film Trustee Company, Ltd."
342,tt0083944,Rambo: First Blood,1982,R,5580,"Anabasis Investments, N.V."
366,tt1454029,The Help,2011,PG-13,8760,"DreamWorks II Distribution Co., LLC"
371,tt4034228,Manchester by the Sea,2016,R,8280,"KFilms Manchester, LLC"
445,tt6107548,Late Night,2019,R,6120,"Late Night Distribution, LLC"
499,tt8400584,The Perfect Find,2023,R,5940,"Confluential Films|AGC Studios|I’ll Have Another|HR Entertainment, Inc."
660,tt1972779,The Best of Me,2014,PG-13,7020,"Best of Me Productions, LLC"
809,tt0892767,Horsemen,2009,R,5400,"Horsemen Productions, LLC"


After a bit of research into the studio for the movie "Abandon", the full name of the studio is 'MFP Munich Film Partners New Century, GMBH & Co. Ham Productions KG', and that is a unique studio so I will leave that one as is. Later on, we will change any unique studios to 'Other' so we don't mess with our analysis later on. Let's update the other studios that needed to be updated. 

In [21]:
wbr = movies[movies['imdb_id'] == 'tt4537896'] 
spider = movies[movies['imdb_id'] == 'tt4633694']
peewee = movies[movies['imdb_id'] == 'tt0837156']

movies.loc[wbr.index, 'studio'] = 'Columbia Pictures|Protozoa Pictures|Studio 8'
movies.loc[spider.index, 'studio'] = 'Columbia Pictures|Sony Pictures|Sony Pictures Animation'
movies.loc[peewee.index, 'studio'] = 'SYOSSET PRODUCTIONS, INC.|PEE-WEE HERMAN PRODUCTIONS, INC.'

In [22]:
selected_ids = ['tt4537896', 'tt4633694', 'tt0837156']
movies[movies['imdb_id'].isin(selected_ids)] 

,imdb_id,name,year,mpaa_rating,duration_sec,studio
288,tt4633694,Spider-Man: Into the Spider-Verse,2018,PG,7020,Columbia Pictures|Sony Pictures|Sony Pictures Animation
1432,tt0837156,Pee-Wee's Big Holiday,2016,PG,5400,"SYOSSET PRODUCTIONS, INC.|PEE-WEE HERMAN PRODUCTIONS, INC."
1631,tt4537896,White Boy Rick,2018,R,6660,Columbia Pictures|Protozoa Pictures|Studio 8


The studios for these three movies are now corrected. We can proceed with looking at the studio column. 

In [23]:
pd.reset_option('display.max_colwidth')
movies

,imdb_id,name,year,mpaa_rating,duration_sec,studio
0,tt11274492,The Out-Laws,2023,R,5700,Happy Madison Productions
1,tt12263384,Extraction 2,2023,R,7380,Filmhaus Films|AGBO
2,tt16419074,Air,2023,R,6720,Mandalay Pictures|Amazon Studios|Skydance Spor...
3,tt14400246,Bird Box Barcelona,2023,TV-MA,7440,Nostromo Pictures|Bluegrass Films|Chris Morgan...
4,tt1745960,Top Gun: Maverick,2022,PG-13,7860,Paramount|Jerry Bruckheimer Films|Don Simpson/...
...,...,...,...,...,...,...
1729,tt6902676,Guns Akimbo,2020,R,5700,Ingenious Media|Occupant Films|Four Knights Fi...
1730,tt3813310,Cop Car,2015,R,5280,Universal
1731,tt2091935,Mr. Right,2016,R,5700,Focus World
1732,tt13372794,The Manor,2021,TV-MA,4860,Amazon Studios|Blumhouse Television


Let's look at the current count of studios as they currently are. 

In [24]:
movies['studio'].describe()

count          1729
unique          934
top       Universal
freq            180
Name: studio, dtype: object

We see there are 934 unique studios, but for a lot of them, that is a unique combination when having multiple studios separated by |'s. Let's see how many studios contain a | which signifies there were multiple studios.

In [25]:
pipe_studios = movies[movies['studio'].str.contains('\|', na=False)]

pipe_studios

,imdb_id,name,year,mpaa_rating,duration_sec,studio
1,tt12263384,Extraction 2,2023,R,7380,Filmhaus Films|AGBO
2,tt16419074,Air,2023,R,6720,Mandalay Pictures|Amazon Studios|Skydance Spor...
3,tt14400246,Bird Box Barcelona,2023,TV-MA,7440,Nostromo Pictures|Bluegrass Films|Chris Morgan...
4,tt1745960,Top Gun: Maverick,2022,PG-13,7860,Paramount|Jerry Bruckheimer Films|Don Simpson/...
6,tt8936646,Extraction,2020,R,6960,India Take One Productions|T.G.I.M. Films|AGBO...
...,...,...,...,...,...,...
1720,tt6662736,What They Had,2018,R,6060,Bona Fide Productions|United Pictures|June Pic...
1727,tt3628574,State Like Sleep,2019,R,6300,3311 Productions|Scythia Films|Code Red Produc...
1728,tt7074886,The Front Runner,2018,R,6780,Bron Studios|Right of Way Films|Creative Wealt...
1729,tt6902676,Guns Akimbo,2020,R,5700,Ingenious Media|Occupant Films|Four Knights Fi...


There are 504 rows with multiple studios, so let's split them out and see how many total studios were included in all these fields. 

In [26]:
piped_studios_concatenated = '|'.join(pipe_studios['studio'].dropna())
unique_studios = set(piped_studios_concatenated.split('|'))
len(unique_studios)

1084

There are 1084 studios hiding between the 504 rows. Let's get a count of each unique studio.

In [27]:
studio_counts = Counter(piped_studios_concatenated.split('|'))

sorted_studio_counts = sorted(studio_counts.items(), key=lambda x: x[1], reverse=True)

studio_df = pd.DataFrame(sorted_studio_counts, columns=['Studio', 'Count'])
studio_df

,Studio,Count
0,Universal Pictures,34
1,Columbia Pictures,26
2,Amazon Studios,25
3,Paramount,20
4,Sony Pictures,15
...,...,...
1079,Altitude Film Entertainment,1
1080,Hyperion Media Group,1
1081,The Electric Shadow Company,1
1082,Maze Pictures,1


We see that the count of each individual studio is relatively low (with a lot of unique studios), but they would still count toward our totals potentially, so it'll still be important to split them out. Let's order alphabetically to see if there are any that could maybe be combined together. 

In [28]:
piped_studios_concatenated = '|'.join(pipe_studios['studio'].dropna())
studio_counts = Counter(piped_studios_concatenated.split('|'))

sorted_studio_counts = sorted(studio_counts.items(), key=lambda x: (x[0]))

sorted_studio_df = pd.DataFrame(sorted_studio_counts, columns=['Studio', 'Count'])

# Filter studios that start with 'A'
filtered_studios = sorted_studio_df[sorted_studio_df['Studio'].str.startswith('Dre')]

filtered_studios.head(40)

,Studio,Count
285,Dreadnought Films,1
286,DreamWorks,2
287,DreamWorks Animation,3
288,DreamWorks Pictures,2
289,Dreamin' Dolphin Film,1


We see that there are some studios that are being counted as unique right now that probably should be combined such as DreamWorks and DreamWorks Pictures. Let's do some string comparison among all studios (piped or not) so we can get them combined. 

We will use some functions from the fuzzywuzzy package to get scores and find similar studios. We will be using a unique threshold of 95 so we only get very close results. First we will split the movies up into the split variable which will contain all movie studios, including both those that are split by pipes, and those that are not. Then we will add them to a dictionary that will contain the studio name, then all the similar studios that were at least 95 percent similar. We will then put the results in a dataframe and display the results. 

In [29]:
pd.set_option('display.max_colwidth', None)
split = movies['studio'].str.split('|', expand=True).stack().unique()

# Create a dictionary to store similar studio names
similar_studio_dict = {}

# Loop through each studio in split
for studio in split:
    similar_studios = find_similar_studios(studio, split)
    if len(similar_studios) >= 1:
        similar_studio_dict[studio] = [similar_studio for similar_studio, _ in similar_studios]

result_df = pd.DataFrame(similar_studio_dict.items(), columns=['Studio', 'Similar Studios'])

result_df.head(40)

,Studio,Similar Studios
0,Happy Madison Productions,[Happy Madison Productions]
1,Filmhaus Films,[Filmhaus Films]
2,AGBO,[AGBO]
3,Mandalay Pictures,[Mandalay Pictures]
4,Amazon Studios,"[Amazon Studios, Amazon]"
5,Skydance Sports,"[Skydance Sports, Skydance]"
6,Artists Equity,[Artists Equity]
7,Nostromo Pictures,[Nostromo Pictures]
8,Bluegrass Films,[Bluegrass Films]
9,Chris Morgan Productions,[Chris Morgan Productions]


With this list, I was able to sort through it and create the mapping of studios that should be grouped together. For example, 'Amazon Studios' had a similar studio to it called 'Amazon'. I compiled these values and sorted through them to determine if there were any that needed to be mapped differently to be more consistent across films. It's possible my mapping is not perfect, but it allows companies under the same umbrella to be represented together. There could be an argument made against doing that, but for this analysis, we will see it through. 

In order to help with the analysis to create the mapping, I created the below code block to help me find specific studios which allowed me to see what the row looks like in regards to different studios. The example below is showing anything that includes "Right of Way Films" as the studio, but can be used with any studio I want to look up. 

In [30]:
pd.reset_option('display.max_colwidth')
check_movies = movies['studio'].str.split('|', expand=True)

target_studio = "Right of Way Films"
studio_found_mask = check_movies.applymap(lambda cell: target_studio in str(cell))

# Find rows where the target studio was found in any of the columns
matching_rows = studio_found_mask.any(axis=1)

# Filter the DataFrame to get the rows where the studio was found
result = check_movies[matching_rows]

result

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
1728,Bron Studios,Right of Way Films,Creative Wealth Media Finance,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


I also could look up the name of the movie in the original dataframe to see what the original data looked like. Here is the movie 'The Front Runner' that is a film that had Right of Way Films as one of the studios. 

In [31]:
movies[movies['name'] == 'The Front Runner'] 

,imdb_id,name,year,mpaa_rating,duration_sec,studio
1728,tt7074886,The Front Runner,2018,R,6780,Bron Studios|Right of Way Films|Creative Wealt...


With the mapping created and stored in the variable studio_mapping (see constants section for the full mapping), let's take a look at all the possible studios, as well as their rating so we can get the values updated. To do this, we will take all the movie studios and expand them in a dataframe where the name of the movies is the index, columns 0-17 are all the studios associated to the movie, and the mpaa_rating of the movie is included as well in the last column. 

In [32]:
split_studios = movies['studio'].str.split('|', expand=True)
split_studios = split_studios.applymap(lambda x: x.strip() if isinstance(x, str) else x)
split_studios['mpaa_rating'] = movies['mpaa_rating']
split_studios.index= movies.name
split_studios

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,mpaa_rating
name,,,,,,,,,,,,,,,,,,,
The Out-Laws,Happy Madison Productions,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Extraction 2,Filmhaus Films,AGBO,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Air,Mandalay Pictures,Amazon Studios,Skydance Sports,Artists Equity,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Bird Box Barcelona,Nostromo Pictures,Bluegrass Films,Chris Morgan Productions,Dylan Clark Productions,None,None,None,None,None,None,None,None,None,None,None,None,None,None,TV-MA
Top Gun: Maverick,Paramount,Jerry Bruckheimer Films,Don Simpson/Jerry Bruckheimer Films,TC Productions,Tencent Pictures,Skydance Media,None,None,None,None,None,None,None,None,None,None,None,None,PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Guns Akimbo,Ingenious Media,Occupant Films,Four Knights Film,Altitude Film Entertainment,Hyperion Media Group,The Electric Shadow Company,Maze Pictures,Pump Metal Films,None,None,None,None,None,None,None,None,None,None,R
Cop Car,Universal,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Mr. Right,Focus World,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R


We then want to iterate through each value and replace anything that is in the mapping. For example, one of the mapping pairs was Jerry Bruckheimer Films and Don Simpson/Jerry Bruckheimer Films. Since these are essentially the same company, the mapping will replace one of them to be the same as the other so they can be grouped together. In this case, Don Simpson/Jerry Bruckheimer Films will be changed to Jerry Bruckheimer Films. 

In [33]:
# Iterate through each row in the DataFrame
for index, row in split_studios.iterrows():
    for col in range(18):  # Columns 0-17
        # Get the value in the current cell
        value = row[col]
        
        # If the value is in the studio_mapping, replace it
        if value in studio_mapping:
            split_studios.at[index, col] = studio_mapping[value]

split_studios

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,mpaa_rating
name,,,,,,,,,,,,,,,,,,,
The Out-Laws,Happy Madison Productions,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Extraction 2,Filmhaus Films,AGBO,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Air,Mandalay Pictures,Amazon Studios,Skydance,Artists Equity,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Bird Box Barcelona,Nostromo Pictures,Bluegrass Films,Chris Morgan Productions,Dylan Clark Productions,None,None,None,None,None,None,None,None,None,None,None,None,None,None,TV-MA
Top Gun: Maverick,Paramount,Jerry Bruckheimer Films,Jerry Bruckheimer Films,TC Productions,Tencent Pictures,Skydance,None,None,None,None,None,None,None,None,None,None,None,None,PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Guns Akimbo,Ingenious Media,Occupant Films,Four Knights Film,Altitude Film Entertainment,Hyperion Media Group,The Electric Shadow Company,Maze Pictures,Pump Metal Films,None,None,None,None,None,None,None,None,None,None,R
Cop Car,Universal Pictures,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Mr. Right,Focus,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R


Now that we have mapped everything, we are going to check the values again to see if there are any duplicates now that we have replaced some studio names. We see one with Top Gun: Maverick with Jerry Bruckheimer Films listed twice, so we will need to handle duplicates like that. If there are any duplicates, we will replace them with None. 

In [34]:
# Create a new DataFrame with the cleaned values
split_studios_cleaned_df = split_studios.apply(clean_duplicates, axis=1)
split_studios_cleaned_df = pd.DataFrame(split_studios_cleaned_df.tolist(), index=split_studios_cleaned_df.index, columns=split_studios.columns)

split_studios_cleaned_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,mpaa_rating
name,,,,,,,,,,,,,,,,,,,
The Out-Laws,Happy Madison Productions,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Extraction 2,Filmhaus Films,AGBO,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Air,Mandalay Pictures,Amazon Studios,Skydance,Artists Equity,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Bird Box Barcelona,Nostromo Pictures,Bluegrass Films,Chris Morgan Productions,Dylan Clark Productions,None,None,None,None,None,None,None,None,None,None,None,None,None,None,TV-MA
Top Gun: Maverick,Paramount,Jerry Bruckheimer Films,None,TC Productions,Tencent Pictures,Skydance,None,None,None,None,None,None,None,None,None,None,None,None,PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Guns Akimbo,Ingenious Media,Occupant Films,Four Knights Film,Altitude Film Entertainment,Hyperion Media Group,The Electric Shadow Company,Maze Pictures,Pump Metal Films,None,None,None,None,None,None,None,None,None,None,R
Cop Car,Universal Pictures,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Mr. Right,Focus,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R


The last thing we need to do before converting these back is take any studios that show up 2 times or less, and replace them as 'Other'. We want to do this for analysis purposes so the system doesn't automatically predict the rating if a movie matches the studio exactly as studios do have the ability to make different ratings for movies. 

In [35]:
#Need to update unique studios to other

# Flatten the DataFrame into a Series and find studio counts
studio_counts = split_studios_cleaned_df.stack().value_counts()

# Replace studios that appear only once with 'Other'
for studio, count in studio_counts.items():
    if count <= 2:
        split_studios_cleaned_df = split_studios_cleaned_df.replace(studio, 'Other')

In [36]:
split_studios_cleaned_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,mpaa_rating
name,,,,,,,,,,,,,,,,,,,
The Out-Laws,Happy Madison Productions,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Extraction 2,Other,AGBO,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Air,Mandalay Pictures,Amazon Studios,Skydance,Other,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Bird Box Barcelona,Nostromo Pictures,Other,Other,Other,None,None,None,None,None,None,None,None,None,None,None,None,None,None,TV-MA
Top Gun: Maverick,Paramount,Jerry Bruckheimer Films,None,Other,Tencent Pictures,Skydance,None,None,None,None,None,None,None,None,None,None,None,None,PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Guns Akimbo,Ingenious Media,Other,Other,Other,Other,Other,Other,Other,None,None,None,None,None,None,None,None,None,None,R
Cop Car,Universal Pictures,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R
Mr. Right,Focus,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,R


One thing to notice is that some movies may have 'Other' filled in multiple times. This would still be valid data based on filling in unique or nearly unique studio names, so we want to keep them as is for this analysis. 

Let's now convert the 'studios' column of our movies dataframe to an updated version with our newly mapped studios. We will separate them with a | again like it was originally and then in our Preprocessing step, we will split out the studios again and create dummy variables. We will also add a column with number of studios to see if that gives any information as well regarding rating. 

The first thing we will do is make a copy of our original movies dataframe. 

In [37]:
cleaned_movies_df = movies.copy()
cleaned_movies_df

,imdb_id,name,year,mpaa_rating,duration_sec,studio
0,tt11274492,The Out-Laws,2023,R,5700,Happy Madison Productions
1,tt12263384,Extraction 2,2023,R,7380,Filmhaus Films|AGBO
2,tt16419074,Air,2023,R,6720,Mandalay Pictures|Amazon Studios|Skydance Spor...
3,tt14400246,Bird Box Barcelona,2023,TV-MA,7440,Nostromo Pictures|Bluegrass Films|Chris Morgan...
4,tt1745960,Top Gun: Maverick,2022,PG-13,7860,Paramount|Jerry Bruckheimer Films|Don Simpson/...
...,...,...,...,...,...,...
1729,tt6902676,Guns Akimbo,2020,R,5700,Ingenious Media|Occupant Films|Four Knights Fi...
1730,tt3813310,Cop Car,2015,R,5280,Universal
1731,tt2091935,Mr. Right,2016,R,5700,Focus World
1732,tt13372794,The Manor,2021,TV-MA,4860,Amazon Studios|Blumhouse Television


Now we will loop through each row of our cleaned up studios dataframe and concatenate anything with mutliple studios with |'s. Lastly, we will add the column "number_of_studios" as another column so we can see how many studios a movie has. 

In [38]:
# Loop through each row in split_studios_cleaned_df
for index, row in split_studios_cleaned_df.iterrows():
    # Convert the non-None row values (excluding 'mpaa_rating') to a list of strings
    row_studios = [str(value) for value in row.drop('mpaa_rating') if value is not None]
    
    # Concatenate the strings with '|' separator
    concatenated_studios = '|'.join(row_studios)
    
    # Get the corresponding indices in cleaned_movies_df using the 'name' value
    movie_indices = cleaned_movies_df.index[cleaned_movies_df['name'] == index]
    
    # Update the 'studio' column in cleaned_movies_df for each matching index
    for movie_index in movie_indices:
        cleaned_movies_df.at[movie_index, 'studio'] = concatenated_studios

# Add column number_of_studios to the data
cleaned_movies_df['number_of_studios'] = cleaned_movies_df['studio'].apply(lambda x: x.count('|') + 1)

# Print the modified cleaned_movies_df DataFrame
cleaned_movies_df

,imdb_id,name,year,mpaa_rating,duration_sec,studio,number_of_studios
0,tt11274492,The Out-Laws,2023,R,5700,Happy Madison Productions,1
1,tt12263384,Extraction 2,2023,R,7380,Other|AGBO,2
2,tt16419074,Air,2023,R,6720,Other|Skydance,2
3,tt14400246,Bird Box Barcelona,2023,TV-MA,7440,Nostromo Pictures|Other|Other|Other,4
4,tt1745960,Top Gun: Maverick,2022,PG-13,7860,Paramount|Jerry Bruckheimer Films|Other|Tencen...,5
...,...,...,...,...,...,...,...
1729,tt6902676,Guns Akimbo,2020,R,5700,Ingenious Media|Other|Other|Other|Other|Other|...,8
1730,tt3813310,Cop Car,2015,R,5280,Universal Pictures,1
1731,tt2091935,Mr. Right,2016,R,5700,Focus,1
1732,tt13372794,The Manor,2021,TV-MA,4860,Amazon Studios|Blumhouse Television,2


We have now cleaned up our movies dataframe and added the number of studios as well. 

### movie_tags <a id='movie_tags'>

Let's now take a look at the movie_tags. This is a dataframe that includes a row for each type of tag in a movie and includes how many times it has occurred in the movie as well as how long it occurred. A movie may have multiple rows if there are multiple tags for that movie. 

In [39]:
movie_tags

,imdb_id,category,tag_name,occurrence_cnt,duration_sec
0,tt0052357,language,blasphemy,1,0
1,tt0052357,violence,non_graphic,5,30
2,tt0052357,violence,disturbing_images,1,0
3,tt0052357,immodesty,immodesty,1,6
4,tt0052357,immodesty,nudity_implied,1,30
...,...,...,...,...,...
23975,tt9902160,violence,non_graphic,9,18
23976,tt9902160,violence,graphic,4,12
23977,tt9902160,immodesty,immodesty,3,30
23978,tt9902160,sexual,sexually_suggestive,1,6


We see that the duration_sec column does have some 0's filled out, but if something exists, it would have at least a 0.1 second duration. As a result, we will update the value to 0.1 so it has a non-zero value for analysis purposes. 

In [40]:
# Replace 0 values with 0.1 in the 'duration_sec' column
movie_tags['duration_sec'] = movie_tags['duration_sec'].replace(0, 0.1)
movie_tags

,imdb_id,category,tag_name,occurrence_cnt,duration_sec
0,tt0052357,language,blasphemy,1,0.1
1,tt0052357,violence,non_graphic,5,30.0
2,tt0052357,violence,disturbing_images,1,0.1
3,tt0052357,immodesty,immodesty,1,6.0
4,tt0052357,immodesty,nudity_implied,1,30.0
...,...,...,...,...,...
23975,tt9902160,violence,non_graphic,9,18.0
23976,tt9902160,violence,graphic,4,12.0
23977,tt9902160,immodesty,immodesty,3,30.0
23978,tt9902160,sexual,sexually_suggestive,1,6.0


Now that we have updated the 0's in the duration_sec column, let's review the datatypes and null values to see if we need to make any other adjustments. 

In [41]:
movie_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23980 entries, 0 to 23979
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imdb_id         23980 non-null  object 
 1   category        23980 non-null  object 
 2   tag_name        23980 non-null  object 
 3   occurrence_cnt  23980 non-null  int64  
 4   duration_sec    23980 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 936.8+ KB


We see there are no null values, and all the data types look correct. Let's see if there are any duplicates to cleanup. 

In [42]:
movie_tags[movie_tags.duplicated()]

,imdb_id,category,tag_name,occurrence_cnt,duration_sec


No duplicates as well. It's time to save our cleaned data. 

# Save the Data <a id='Save_the_Data'>

Now that we have finished cleaning our data, we will save our cleaned up dataframes so they are ready to use for our  exploratory data analysis, preprocessing, and modeling steps. 

In [43]:
#Save the modified dataframe to a new .csv file called clean_data.csv
tags.to_csv("data/clean_data/cleaned_tags.csv", index=False)
cleaned_movies_df.to_csv("data/clean_data/cleaned_movies.csv", index=False)
movie_tags.to_csv("data/clean_data/cleaned_movie_tags.csv", index=False)
split_studios_cleaned_df.to_csv("data/clean_data/split_studios_cleaned_df.csv", index=False)

In this notebook, we were able to cleanup our data by handling null values, removing duplicates, and handling specific scenarios in regards to studios for our analysis. We are now ready to move to Exploratory Data Analysis to find some interesting insights in our data which we will do in the next notebook. 